In [ ]:
# FFT Function
# Adjust to fit problem values and equations

def myfft(xlow, xhigh, barwidth=0.9):
    N = 300 # Spacing
    T = 1/N
    
    x = np.linspace(xlow, xhigh, N, endpoint=False)
    y = 2*np.sin(2.0*np.pi*x/7) - 4*np.sin(3.0*np.pi*x/5)
    
    fig, ax = plt.subplots(1,2)
    fig.set_size_inches([9, 5])
    ax[0].plot(x,y)
    ax[0].set_xlabel('x')
    ax[0].set_ylabel('y')
    
    amp = abs(np.fft.fft(y))[:N//2]
    freq = np.fft.fftfreq(N,T)[:N//2]
    
    ax[1].bar(freq, 2*amp/N, width=barwidth)
    ax[1].plot(freq, 2*amp/N, 'r*')
    ax[1].set_xlabel('Frequency')
    ax[1].set_ylabel('Amplitude')
    ax[1].set_xlim(0,30)
    
    ax[0].set_title('Wave Function')
    ax[1].set_title('Frequency vs. Amplitude')
    
    return

def fftNoise(df, buckets=4096):
    
    # Size of frequency buckets: 4096
    buckets = buckets
    
    line_count = df.shape[0]
    po = 2e-5 #Pa
    
    # Total time for the noise data 5s
    sec = 5
    
    ensembles = df.shape[0] //buckets  # Chunk size
    sec_b = sec / ensembles  # Time period
    
    print(f"Frequency buckets size: {buckets}")
    print(f"Number of ensembles: {ensembles}")
    print(f"Time period: {sec_b}s")
    
    dfEnsembles = np.array_split(df, ensembles)  # Split to ensemble
    
    p1f = np.array([abs(np.fft.fft(dfEnsembles[i], axis=0)) for i in range(ensembles)])  # DFT and magnitude
    p1f_t = np.sum((2*p1f/buckets)**2, axis=0)  # Average all the ensembles
    p1f_t = np.sqrt(p1f_t/ensembles)  # RMS
    
    db = 20*np.log10(p1f_t[:buckets//2]/po)  # SPL
    
    t = np.linspace(0, sec_b, buckets)  # Total time for plotting
    tf = np.fft.fftfreq(buckets, sec_b/(buckets - 1))[:buckets//2]  # Convert to frequencies for plotting
    
    # Plot 1, 4, 13
    
    fig, ax = plt.subplots()
    fig.set_size_inches([15,10])
    ax.plot(tf, db[:,0], label='Mic 1')
    ax.plot(tf, db[:,3], label='Mic 4')
    ax.plot(tf, db[:,12], label='Mic 13')
    ax.set_ylabel('Frequency (Hz)')
    ax.set_xlabel('SPL (dB)')
    ax.set_xscale('log')
    ax.set_xlim([100, 1e5])
    ax.grid(which='minor', ls=':')
    ax.grid(which='major')
    ax.legend()
    plt.show()
    
    return tf, db